In [ ]:
!pip install -r ../../requirements.txt

In [1]:
import path
path.setPathLocalNotebook()

from diffuserslib import loginHuggingFace, DiffusersPipelines, DiffusersBaseModel, GenerationParameters, ModelParameters

# loginHuggingFace('token')

pipelines = DiffusersPipelines(device = "mps", safety_checker=True, cache_dir=None)
pipelines.loadPresetFile("../../modelconfig.yml")

print(pipelines.presets.getModelsByType("controlimage"))
# pipelines.createTextToImagePipeline('kandinsky-community/kandinsky-2-1-prior')

# pipelines.loadLORA("epiNoiseoffset_v2.safetensors", "sd_1_5", weight=1.5)

# First-time "warmup" pass because of issue with m1 cpu: https://github.com/huggingface/diffusers/issues/372
# _, _ = diffusers.textToImage(prompt="", negprompt="", steps=5, scale=7, width=512, height=512)



{'lllyasviel/control_v11p_sd15_canny': DiffusersModel(modelid='lllyasviel/control_v11p_sd15_canny', base='sd_1_5', modeltype='controlnet', pipelinetypes={}, revision=None, stylephrase=None, vae=None, autocast=True, location='hf', modelpath='lllyasviel/control_v11p_sd15_canny', data={'id': 'lllyasviel/control_v11p_sd15_canny'}), 'lllyasviel/control_v11f1p_sd15_depth': DiffusersModel(modelid='lllyasviel/control_v11f1p_sd15_depth', base='sd_1_5', modeltype='controlnet', pipelinetypes={}, revision=None, stylephrase=None, vae=None, autocast=True, location='hf', modelpath='lllyasviel/control_v11f1p_sd15_depth', data={'id': 'lllyasviel/control_v11f1p_sd15_depth'}), 'lllyasviel/control_v11p_sd15_inpaint': DiffusersModel(modelid='lllyasviel/control_v11p_sd15_inpaint', base='sd_1_5', modeltype='controlnet', pipelinetypes={}, revision=None, stylephrase=None, vae=None, autocast=True, location='hf', modelpath='lllyasviel/control_v11p_sd15_inpaint', data={'id': 'lllyasviel/control_v11p_sd15_inpaint'

In [ ]:
from IPython.display import display

prompt = "a photo of an astronaut riding a horse on mars, dark, dimly lit"
model = "runwayml/stable-diffusion-v1-5"
# model = "kandinsky-community/kandinsky-2-1"

params = GenerationParameters(prompt=prompt, negprompt="", steps=40, cfgscale=7, width=512, height=512, scheduler="EulerDiscreteScheduler", seed=None, models=[ModelParameters(model)])
image, seed = pipelines.generate(params)

print(f"Seed: {seed}")

# for image in images:
display(image)


In [ ]:
from IPython.display import display
from PIL import Image

prompt = "a photo of an astronaut riding a horse on mars, dark, dimly lit"
# model = "stabilityai/stable-diffusion-2-1-base"
model = "runwayml/stable-diffusion-v1-5"
controlmodel = "lllyasviel/sd-controlnet-canny"

image = Image.open("data/rabbit.png")
control = Image.open("data/rabbit.png")
image, seed = pipelines.imageToImageControlNet(prompt=prompt, initimage=image, controlimage=control, 
                                               negprompt="", strength=0.5, scale=7, width=512, height=512, scheduler="EulerDiscreteScheduler", seed=None, 
                                               model=model, controlmodel=controlmodel)

print(f"Seed: {seed}")

# for image in images:
display(image)

In [ ]:
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base")
pipe = pipe.to("mps")

# Recommended if your computer has < 64 GB of RAM
# pipe.enable_attention_slicing()

prompt = "a photo of an astronaut riding a horse on mars"

# First-time "warmup" pass (see explanation above)
_ = pipe(prompt, num_inference_steps=1)

# Results match those from the CPU device after the warmup pass.
image = pipe(prompt).images[0]